---
title: Mission Pipeline
subtitle: Mission impossible
description: Mission pipeline process data from a specific mission.
---

Generally, it includes the following steps:

- combine features from different sources/instruments (magnetic field, state data, etc.)
- generate new features

Additional components:

- `create_events_pipeline` | `create_candidate_pipeline` 
    - `create_sw_events_pipeline` : (Optional) pipeline for solar wind candidates (used in mission `JUNO` and `ARTEMIS`) to exclude events that may be influenced by planetary magnetosphere.



In [ ]:
#| default_exp pipelines/default/mission

In [ ]:
#| export
import polars as pl

from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

## Combine features

In [ ]:
# | export
def combine_features(candidates: pl.LazyFrame, states_data: pl.LazyFrame):
    vec_cols = ["v_x", "v_y", "v_z"]  # plasma velocity vector in any coordinate system
    b_vecL_cols = ["Vl_x", "Vl_y", "Vl_z"]  # major eigenvector in any coordinate system
    if not set(vec_cols).issubset(states_data.columns):
        raise ValueError(f"Missing columns {vec_cols}")
    if not set(b_vecL_cols).issubset(candidates.columns):
        raise ValueError(f"Missing columns {b_vecL_cols}")

    return candidates.sort("time").join_asof(states_data.sort("time"), on="time")

## Combining magnetic field data and state data

With combined dataset, we calculate additional features for each candidate.

$$L_{mn} = v_{mn}  T_{duration}$$

$$ j_0 = (\frac{d B}{d t})_{max} \frac{1}{v_{mn}}$$

### Calculating additional features for the combined dataset

In [ ]:
#| export
import astropy.units as u
from astropy.constants import mu0, e
from plasmapy.formulary.lengths import inertial_length
from plasmapy.formulary.speeds import Alfven_speed
from xarray_einstats import linalg
from ids_finder.utils.basic import df2ts, pl_norm
import xarray as xr

In [ ]:
#| export
def vector_project(v1,v2, dim="v_dim"):
    return xr.dot(v1 , v2, dims=dim) / linalg.norm(v2, dims=dim)

def vector_project_pl(df: pl.DataFrame, v1_cols, v2_cols, name=None):
    
    v1 = df2ts(df, v1_cols).assign_coords(v_dim=["r","t","n"])
    v2 = df2ts(df, v2_cols).assign_coords(v_dim=["r","t","n"]) 
    result = vector_project(v1, v2, dim="v_dim")
    
    return df.with_columns(
        pl.Series(result.data).alias(name or "v_proj")
    )

### Inertial length

In [ ]:
# | export
def compute_inertial_length(ldf: pl.LazyFrame, density_col = "plasma_density"):
    df = ldf.collect()

    density = df[density_col].to_numpy() * u.cm ** (-3)
    result = inertial_length(density, "H+").to(u.km)

    return df.with_columns(ion_inertial_length=pl.Series(result.value)).lazy()

### Alfven current

In [ ]:
#| export
def compute_Alfven_speed(ldf: pl.LazyFrame):
    df = ldf.collect()

    B = df["B"] if "B" in df.columns else df["b_mag"]  # backwards compatiblity
    density = df["plasma_density"].to_numpy() * u.cm ** (-3)
    result = Alfven_speed(B.to_numpy() * u.nT, density=density, ion="p+").to(u.km / u.s)

    return df.with_columns(Alfven_speed=pl.Series(result.value)).lazy()


def compute_Alfven_current(ldf: pl.LazyFrame):
    df = ldf.collect()

    Alfven_speed = df["Alfven_speed"].to_numpy() * u.km / u.s
    density = df["plasma_density"].to_numpy() * u.cm ** (-3)

    result = (e.si * Alfven_speed * density)
    result = result.to(u.nA / u.m**2)

    return df.with_columns(j_Alfven=pl.Series(result.value)).lazy()

### Pipelines

In [ ]:
# | export
def calc_combined_features(df: pl.LazyFrame):
    vec_cols = ["v_x", "v_y", "v_z"]  # plasma velocity vector in any coordinate system
    b_vecL_cols = ["Vl_x", "Vl_y", "Vl_z"]  # major eigenvector in any coordinate system

    j_factor = ((u.nT / u.s) * (1 / mu0 / (u.km / u.s))).to(u.nA / u.m**2)

    result = (
        df.with_columns(
            duration=pl.col("d_tstop") - pl.col("d_tstart"),
        )
        .pipe(vector_project_pl, vec_cols, b_vecL_cols, name="v_l")
        .with_columns(v_mn=(pl.col("plasma_speed") ** 2 - pl.col("v_l") ** 2).sqrt())
        .with_columns(
            L_mn=pl.col("v_mn") * pl.col("duration").dt.nanoseconds() / 1e9,
            j0=pl.col("d_star") / pl.col("v_mn"),
        )
        .pipe(compute_inertial_length)
        .pipe(compute_Alfven_speed)
        .pipe(compute_Alfven_current)
        .pipe(j0=pl.col("j0") * j_factor.value)
        .with_columns(
            L_mn_norm=pl.col("L_mn") / pl.col("ion_inertial_length"),
            j0_norm=pl.col("j0") / pl.col("j_Alfven"),
        )
    )
    return result

## Pipelines

In [ ]:
#| export
def create_combined_data_pipeline(
    sat_id, 
    tau: int = 60,
    ts_mag: int = 1,
    ts_state: str = "1h",
    **kwargs) -> Pipeline:

    ts_mag_str = f"ts_{ts_mag}s"
    tau_str = f"tau_{tau}s"

    node_combine_features = node(
        combine_features,
        inputs=[
            f"{sat_id}.feature_{ts_mag_str}_{tau_str}",
            f"{sat_id}.primary_state_{ts_state}",
        ],
        outputs=f"{sat_id}.features",
    )
    
    node_calc_new_features = node(
        calc_combined_features,
        inputs=f"{sat_id}.features",
        outputs=f"candidates.{sat_id}_{ts_mag_str}_{tau_str}",
    )

    nodes = [node_combine_features, node_calc_new_features]
    return pipeline(nodes)